# Ajuste de hiperparametros con agrupamientos

# Random forest

> **El parámetro principal a sintonizar un bosque aleatorio es `n_estimators`.** \
- En general, cuantos más árboles en el bosque, mejor será el rendimiento de la generalización.
- Sin embargo, ralentizará el tiempo de ajuste y predicción.
- El objetivo es equilibrar el tiempo de computación y el rendimiento de la generalización al establecer el número de estimadores al poner dicho modelo en producción.

> Por tanto, también podríamos **sintonizar un parámetro que controla la profundidad de cada árbol en el bosque.**
- Dos parámetros son importantes para esto: `max_depth` y `max_leaf_nodes`.
- Difieren en la forma en que controlan la estructura del árbol.
- De hecho, max_depth reforzará tener un árbol más simétrico, mientras que max_leaf_nodes no impone dicha restricción.

> ⚠ Tener cuidado con el hecho de que en un bosque aleatorio, los árboles son generalmente profundos ya que buscamos sobreajustar cada árbol en cada muestra en cada boostrap, ya que esto se mitigará combinándolos todos.
> - Agrupando de árboles subajustados (es decir, árboles poco profundos) también puede conducir a un bosque subajustado.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data, target = fetch_california_housing(return_X_y=True, as_frame=True)
target *= 100
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0)

In [ ]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

param_distributions = {
    "n_estimators": [1, 2, 5, 10, 20, 50, 100, 200, 500],
    "max_leaf_nodes": [2, 5, 10, 20, 50, 100],
}
search_cv = RandomizedSearchCV(
    RandomForestRegressor(n_jobs=2), param_distributions=param_distributions,
    scoring="neg_mean_absolute_error", n_iter=10, random_state=0, n_jobs=2,
)
search_cv.fit(data_train, target_train)

columns = [f"param_{name}" for name in param_distributions.keys()]
columns += ["mean_test_error", "std_test_error"]
cv_results = pd.DataFrame(search_cv.cv_results_)
cv_results["mean_test_error"] = -cv_results["mean_test_score"]
cv_results["std_test_error"] = cv_results["std_test_score"]
cv_results[columns].sort_values(by="mean_test_error")

- Ahora estimaremos el rendimiento de la generalización del mejor modelo reajustando con el conjunto de capacitación completo y utilizando el conjunto de pruebas para la puntuación en datos no procesados.
    - Esto se hace de forma predeterminada al llamar al método `.fit`.

In [ ]:
error = -search_cv.score(data_test, target_test)
print(f"En promedio, nuestro regresor de bosque aleatorio comete un error de {error:.2f} k$")

# Gradient-boosting decision trees
> Para gradient-boosting **los parámetros están acoplados, por lo que no podemos establecer los parámetros uno tras otro**.
- Los parámetros importantes son `n_estimators`, `learning_rate` y, `max_depth` o `max_leaf_nodes` (como se discutió anteriormente en el bosque aleatorio).

> **max_depth (o max_leaf_nodes)**: El árbol utilizado en el refuerzo de gradiente debe tener una profundidad baja, típicamente entre 3 y 8 niveles, o pocas hojas (2^3 = 8 a 2^8 = 256).
- Tener modelos muy débiles en cada paso ayudará a reducir el sobreajuste.

> Con esto en mente, **cuanto más profundo sean los árboles, más rápido se corregirán los residuos y se requerirán menos mdelos**.
- Por lo tanto, los `n_estimators` deben aumentarse si `max_depth` es más bajo.

> Con una **tasa de aprendizaje muy baja** necesitaremos más estimadores para corregir el error general.
- Sin embargo, una **tasa de aprendizaje demasiado grande** tiende a obtener un conjunto sobreajustado, similar a tener una profundidad de árbol demasiado grande.

In [ ]:
from scipy.stats import loguniform
from sklearn.ensemble import GradientBoostingRegressor

param_distributions = {
    "n_estimators": [1, 2, 5, 10, 20, 50, 100, 200, 500],
    "max_leaf_nodes": [2, 5, 10, 20, 50, 100],
    "learning_rate": loguniform(0.01, 1),
}
search_cv = RandomizedSearchCV(
    GradientBoostingRegressor(), param_distributions=param_distributions,
    scoring="neg_mean_absolute_error", n_iter=20, random_state=0, n_jobs=2
)
search_cv.fit(data_train, target_train)

columns = [f"param_{name}" for name in param_distributions.keys()]
columns += ["mean_test_error", "std_test_error"]
cv_results = pd.DataFrame(search_cv.cv_results_)
cv_results["mean_test_error"] = -cv_results["mean_test_score"]
cv_results["std_test_error"] = cv_results["std_test_score"]
cv_results[columns].sort_values(by="mean_test_error")

In [ ]:
# Now we estimate the generalization performance of the best model using the test set.
# estimamos el rendimiento de generalización del mejor modelo utilizando el conjunto de pruebas.

error = -search_cv.score(data_test, target_test)
print(f"En promedio, nuestro regresor GBDT comete un error de {error:.2f} k$")


- La puntuación media de la prueba en el conjunto de pruebas de retención es ligeramente mejor que la puntuación del mejor modelo.
- La razón es que el modelo final está reajustado en todo el conjunto de entrenamiento y, por lo tanto, en más datos que los modelos internos cros-validados del procedimiento de búsqueda de cuadrícula.